In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import LSTM,Dense, Concatenate,Bidirectional,Input
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras.layers import AdditiveAttention
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec


In [2]:
#importing vocabulary
ds_path='/home/al/Desktop/nnfl/word2vecmod.txt'
data = np.loadtxt(ds_path,dtype=str,usecols=(0))
print(data)
print(data.shape[0])

['a' 'abandon' 'ability' ... '<start>' '<end>' '<pad>']
5035


In [3]:
#loading word vectors for getting unkown words
tmp_file = get_tmpfile(datapath("/home/al/Desktop/nnfl/new"))
word_model = KeyedVectors.load(tmp_file)

In [4]:
#list of words in word vectors
data2= np.loadtxt('/home/al/Desktop/nnflproj/glove.6B.300d.txt',dtype=str,usecols=(0))

In [5]:
#making word-to-index and index-to-word dictionaries
word_to_ix=dict(zip(data,range(0,data.shape[0])))
ix_to_word=dict(zip(range(0,data.shape[0]),data))

In [6]:
#constants
voc=data.shape[0]#for <start>,<end>,<pad>
ip_len=10
op_len=10
n_s=64+op_len#no of decoder lstm
n_a=64#no 


In [7]:
#one-hot function
def oh(Xq,ip_ln):
    Xoh=np.zeros([Xq.shape[0],ip_ln,voc])
    for i in range(Xq.shape[0]):
        for j in range(ip_ln):
            Xoh[i,j,int(Xq[i,j])]=1
    return Xoh

In [8]:
#model
enc_ip=Input(shape=(ip_len,voc))
enc_lstm=LSTM(64,return_state=True,return_sequences=True)
bi=Bidirectional(enc_lstm)
enc_op,h1,c1,h2,c2=bi(enc_ip)
concat=Concatenate(axis=-1)
h=concat([h1,h2])
c=concat([c1,c2])
enc_states=[h,c]
dec_ip=Input(shape=(op_len,voc))
dec_lstm=LSTM(128,return_sequences=True,return_state=True)
dec_op,_,_=dec_lstm(dec_ip,initial_state=enc_states)
att=AdditiveAttention()([dec_op, enc_op])
dense=Dense(voc-1,activation='softmax')
op=concat([dec_op,att])
op=dense(op)
model=Model([enc_ip,dec_ip],op)
model.compile(optimizer="rmsprop",loss='categorical_crossentropy',metrics=['accuracy'])
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 10, 5035)]   0                                            
__________________________________________________________________________________________________
bidirectional (Bidirectional)   [(None, 10, 128), (N 2611200     input_1[0][0]                    
__________________________________________________________________________________________________
concatenate (Concatenate)       multiple             0           bidirectional[0][1]              
                                                                 bidirectional[0][3]              
                                                                 bidirectional[0][2]              
                                                                 bidirectional[0][4]          

In [9]:
#creating checkpoint
filepath="/home/al/Desktop/nnfl final/weightsfinv6.hdf5"
checkpoint=ModelCheckpoint(filepath, monitor='accuracy',verbose=1, save_best_only=True, mode='max')
model.load_weights(filepath)

In [11]:
d_path='/home/al/Desktop/nnfl final/nds-a'
idx=0
idx2=0
for e  in 'a':
#bcdefghi':

    for f  in 'a':
        #bcdefghijklmnopqrstuvwxyz':
        ds = np.loadtxt(d_path+e+f+'.csv',delimiter='\n',dtype='str')
        num_sen=ds.shape[0]
        idx=0
        Xtr=np.ndarray(shape=[num_sen,ip_len])
        for j in range(0,num_sen):
                #ds.shape[0]-(ds.shape[0]%1000)+1)
                w1=np.array(ds[j].lower().split())
                for i in range(0,ip_len):    
                    w=w1[i]
                    if i==ip_len-1:
                        #w1_sum/=50
                        Xtr[j,i]=voc-2#for<end>
                        
                        break
                    if w=='<start>':
                        Xtr[j,i]=voc-3
                        continue
                    if w=='<end>':
                        Xtr[j,i]=voc-2#<end>
                        i+=1
                        while i<ip_len:
                            Xtr[j,i]=voc-1#padding
                            i+=1
                        break
                    if w in data:                
                        Xtr[j,i]=word_to_ix[w]
                    else:
                        try:
                            temp=word_model.most_similar(positive=[w],negative=[],restrict_vocab=4095)#for<unk>
                            idx2+=1
                            Xtr[j,i]=word_to_ix[temp[0][0]]
                        except KeyError:
                            Xtr[j,i]=word_to_ix['it']
                            idx+=1


                        
                           
                if j%1000==0:
                    print(j)
    print(idx)
    print(idx2)


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
4322
109074


In [12]:
b=Xtr[0]
for el in b.flatten():
        print(ix_to_word[el])
print(ds[0])

<start>
you
see
,
we
threat
engaged
.
<end>
<pad>
<start> you see , we re engaged . <end>


In [14]:
#train
model.load_weights(filepath)
for kl in range(128,num_sen,128):
    u=oh(Xtr[kl-128:kl+1],ip_len)
    u=tf.convert_to_tensor(u)
    
    model.fit([u[0:-1],u[0:-1]],u[1:,:,:-1],epochs=10,batch_size=51,use_multiprocessing=True,callbacks=[checkpoint],verbose=2)

model.load_weights(filepath)
print('\n\n\n\n\ndone')

Train on 128 samples
Epoch 1/10

Epoch 00001: accuracy improved from -inf to 0.24453, saving model to /home/al/Desktop/nnfl final/weightsfinv6.hdf5
128/128 - 5s - loss: 2.7916 - accuracy: 0.2445
Epoch 2/10

Epoch 00002: accuracy improved from 0.24453 to 0.25781, saving model to /home/al/Desktop/nnfl final/weightsfinv6.hdf5
128/128 - 1s - loss: 2.4268 - accuracy: 0.2578
Epoch 3/10

Epoch 00003: accuracy improved from 0.25781 to 0.27891, saving model to /home/al/Desktop/nnfl final/weightsfinv6.hdf5
128/128 - 2s - loss: 2.2598 - accuracy: 0.2789
Epoch 4/10

Epoch 00004: accuracy improved from 0.27891 to 0.28672, saving model to /home/al/Desktop/nnfl final/weightsfinv6.hdf5
128/128 - 2s - loss: 2.1602 - accuracy: 0.2867
Epoch 5/10

Epoch 00005: accuracy improved from 0.28672 to 0.28750, saving model to /home/al/Desktop/nnfl final/weightsfinv6.hdf5
128/128 - 2s - loss: 2.1028 - accuracy: 0.2875
Epoch 6/10

Epoch 00006: accuracy improved from 0.28750 to 0.30469, saving model to /home/al/Desk

In [26]:
#test
model.load_weights(filepath)
ut=oh(Xtr[0:1],ip_len)
a=model.predict([ut,ut])
b=np.argmax(a,axis=-1)
print(b)
for el in b.flatten():
        print(ix_to_word[el])
ds[1]

[[5032 5030 5023 2290 5023 5023 5023 5033 5033 5033]]
<start>
i
,
sad
,
,
,
<end>
<end>
<end>


'<start> exactly two years ago today , she and i buried a time capsule here . <end>'

In [ ]:
#if any changes to vocabulary

In [85]:
l1=[]
l2=[]
for i in range(data2.shape[0]):
    if data2[i] in ['are']:
        l1+=[i]
    else:
        l2+=[i]

In [13]:
data4=np.loadtxt('/home/al/Desktop/nnflproj/glove.6B.300d.txt',dtype=float,usecols=(range(1,301)),skiprows=1)

In [97]:
word_to_ix2=dict(zip(data2,range(0,data2.shape[0])))
ix_to_word2=dict(zip(range(0,data2.shape[0]),data2))
a1=data4[word_to_ix2['is']]-data4[word_to_ix2['do']]+data4[word_to_ix2['did']]

In [26]:
#creating vector for had
is11=data4[word_to_ix2['has']]+data4[word_to_ix2['did']]-data4[word_to_ix2['do']]

In [18]:
word_model.add(entities=data2[1765],weights=data4[1765],replace=True)

In [16]:
word_model.save('/home/al/Desktop/nnfl/new')

In [27]:
print(is11 )

[-0.0766063   0.20644    -0.034352   -0.065342    0.27387    -0.664119
  0.122819   -0.45537     0.0077661  -1.8518      0.626968   -0.0391655
 -0.335291    0.52238    -0.0333704   0.385045   -0.172245   -0.007367
 -0.431167   -0.343461   -0.00482     0.17623    -0.201768   -0.1905209
 -0.73655     0.134355   -0.44053    -0.41845    -0.38986     0.69412
  0.02889     0.12712     0.1865      0.470125   -1.14312    -0.92646
 -0.051844   -0.02484     0.35317    -0.194072   -0.1537073   0.149239
 -0.200314    0.35766    -0.0944128   0.469116   -0.71202    -0.355762
 -0.0886521   0.43724    -0.081358   -0.23484     0.282712    0.515287
  0.371574   -0.38545     0.627861    0.48390213  0.565339   -0.62596
 -0.584631    1.044703    0.125403   -0.73778    -0.0633149  -0.75485
  0.005574    0.12999    -0.263811   -0.327531    0.722245   -0.144288
  0.548239    0.275       0.21818     0.228203    0.382868    0.1432693
 -0.6503      0.92362    -0.230969    0.1219582   0.05781     0.376401
  0.554